Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

https://developers.google.com/machine-learning/guides/text-classification

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./whiskey-reviews-dspt4/train.csv')
test = pd.read_csv('./whiskey-reviews-dspt4/test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [3]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts()

1    2881
0    1141
2      65
Name: ratingCategory, dtype: int64

In [4]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
3631,4154,"\nThis Highlander is Lowland-ish in style, but with more heft. There’s soft barley sweetness, notes of vanilla and orange creamsicle. Grassy notes emerge from time to time. It is clean and very drinkable. An ideal aperitif, but could be enjoyed anytime between dawn and dinner. Whoever said whisky was an after-dinner drink never tasted this whisky. \r\n",0
834,4922,"\nHands up if you’d ever wondered what would happen if you seasoned old bourbon barrels with fermenting Irish cider, then added a triple blend of whiskeys? Anyone? Fizzy sherbet, green foliage, and cider (not apple) notes, that’s what. Seasoning suggests the cask occupants are less than good mates, the cider more a lingering tenant. Pot still surfaces through the saccharine cider flavors, with coiled Bramley apple peels, citrus strands, and a nippy spiciness. Just because you can, doesn’t mean you should. €54",0
2943,3850,"\nWemyss Malts names the sources of these single cask bottlings for the first time in the series. ‘Caribbean Fruits’ is a 1990 Glencadam. The hogshead in which it was matured has yielded just 320 bottles. Sweet on the nose, with apricots, fudge, creamy vanilla, and drinking chocolate. Tropical fruits on the palate, with malt, hazelnuts, and spicy oak. Tannins and cocoa powder in the finish. £80",0


In [5]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
2906,5045,"\nMade from 100% rye, but aged in wine casks. Distinctly pink tinge: rosé whiskey. Very young nose: rye grain, Starburst candies, and feinty vegetable notes along with the wine-oak. Thin, dry, reminiscent of a dry hay barn: dusty grain, warm wood planks, a bit medicinal. Aged 130 days, and tastes like it. Growing up in Pennsylvania, Highspire was a town we stayed out of. Good advice.",2
1252,5053,"\nSo that’s what wheat smells like: not much. The aromas here seem to be mostly yeast-derived, with a faint ripe fruitiness (undefined: peach, apple?) and alcohol heat, and a touch of wheat-origin sweet grass. Wheat’s in the mouth, like chewing fresh grain with some water and alcohol. It’s all there, and the finish is sweet, and it’s clean, but…that’s about all. White bread whiskey.",2
1209,5092,"\nThe sherry is very dominant and cloying, which is unfortunate. And I’m not crazy about the quality of the sherry (or perhaps even the wood it was aged in). I have great respect for both Highland Park and Binny’s, but this is somewhat disappointing for a Highland Park. Tasted twice, with the same opinion. (Bottled for Binny’s Beverage Depot)",2


### Split the Training Set into Train/Validation

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

### Define Pipeline Components

In [8]:
vector = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
classifier = RandomForestClassifier()

vect = vector
clf = classifier

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [12]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:   13.2s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [13]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [14]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [15]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [16]:
subNumber = 0

In [17]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

In [18]:
vector = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
classifier = RandomForestClassifier()

vect = vector
clf = classifier

pipe = Pipeline([('vect', vect),
                 ('clf', clf)])

parameters = {
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 10000),
    'clf__n_estimators': (100, 500),
    'clf__max_depth': (10, 20, None)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  3.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [19]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1


In [20]:
print(grid_search.best_score_)
grid_search.best_params_

0.7430466770632835


{'clf__max_depth': None,
 'clf__n_estimators': 500,
 'vect__max_df': 0.7,
 'vect__max_features': 5000,
 'vect__min_df': 2}

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [21]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD

In [46]:
# Use parameters from RandomSearch in previous section

"""'clf__max_depth': None,
 'clf__n_estimators': 500,
 'vect__max_df': 0.7,
 'vect__max_features': 5000,
 'vect__min_df': 2
"""
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2, 
                       max_df=0.7,
                       max_features=5000)

# Add dimensionality reduction
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10,
                   n_components=100)

#lsi = ...

# Use parameters from RandomSearch in previous section
clf = RandomForestClassifier(n_estimators=500, 
                             random_state=42)

params = {
    'svd__n_components': stats.randint(100, 500),
}


pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', clf)         # RandomForest Classifier
])

# Fit
random_search = RandomizedSearchCV(pipe, 
                                   params, 
                                   cv=2, 
                                   #n_iter=3, 
                                   n_jobs=-1, 
                                   verbose=1)

random_search.fit(X_train, y_train)

"""pipe = Pipeline([('lsi', lsi), 
                 ('clf', clf),
                ('vect', vect)])"""

"""parameters = {
    'lsi__svd__n_components': [10,100,250],
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}
"""

"""
From Lecture

pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', rfc)         # RandomForest Classifier
])

params = {
    'svd__n_components': stats.randint(100, 500),
}

"""



Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   24.0s finished


"\nFrom Lecture\n\npipe = Pipeline([\n    ('vect', vect),      # TF-IDF Vectorizer\n    ('svd', svd),        # Truncated SVD Dimensionality Reduction\n    ('clf', rfc)         # RandomForest Classifier\n])\n\nparams = {\n    'svd__n_components': stats.randint(100, 500),\n}\n\n"

In [47]:
print(random_search.best_score_)
random_search.best_params_

0.7060265609618243


{'svd__n_components': 286}

### Make a Submission File

In [41]:
# Predictions on test sample
pred = random_search.predict(test['description'])

In [42]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [43]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [48]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [49]:
# Continue Word Embedding Work Here

import spacy
nlp = spacy.load("en_core_web_lg")

In [50]:
sample = X_train[0]
sample

'\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

In [52]:
doc = nlp(sample)
nlp_vector = doc.vector
print(len(nlp_vector))

300


In [ ]:
nlp_vector

In [54]:
# function to get vector of every line of text from document

def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [55]:
x_train_vectors = get_word_vectors(X_train)
len(x_train_vectors) == len(X_train)

True

In [56]:
x_test_vectors = get_word_vectors(X_test)
len(x_test_vectors) == len(X_test)

True

In [57]:
# fit to the classifeir using the vectors just created

clf = RandomForestClassifier(n_estimators=500, 
                             random_state=42,
                            )

clf.fit(x_train_vectors, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [60]:
# evaluating on test data

from sklearn.metrics import accuracy_score

y_test_pred = clf.predict(x_test_vectors)
accuracy_score(y_test, y_test_pred)

0.7420537897310513

In [67]:
test_vectors = get_word_vectors(test['description'])


### Make a Submission File

In [68]:
# Predictions on test sample
pred = clf.predict(test_vectors)

In [69]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [70]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

In [82]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', 
                    alpha=1e-5,
                    hidden_layer_sizes=(16, 2),
                    random_state=1,
                   max_iter=10000)

In [83]:
clf.fit(x_train_vectors, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(16, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [84]:
# Evaluate on test data
y_test_predic = clf.predict(x_test_vectors)
accuracy_score(y_test, y_test_predic)

0.6968215158924206

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?